In [1]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9994'
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

2023-06-15 00:48:08.342149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[2023-06-15 00:48:11,144] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
torch.manual_seed(42)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
training_args = TrainingArguments(output_dir='./results', num_train_epochs=12, logging_steps=50, save_strategy=IntervalStrategy.NO,
                                  per_device_train_batch_size=15, per_device_eval_batch_size=15, warmup_steps=50,
                                 weight_decay=0.01, logging_dir='./logs', fp16=True, deepspeed='./ds_config.json')
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m").cuda()
model.resize_token_embeddings(len(tokenizer))
descriptions = pd.read_csv('../data.text')['text']
max_length = max([len(tokenizer.encode(description)) for description in descriptions])
print("Max length: {}".format(max_length))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[2023-06-15 00:48:14,487] [WARNING] [comm.py:152:init_deepspeed_backend] NCCL backend in DeepSpeed not yet implemented
[2023-06-15 00:48:14,488] [INFO] [comm.py:594:init_distributed] cdb=None
[2023-06-15 00:48:14,488] [INFO] [comm.py:625:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
Max length: 411


In [4]:
class PersonalDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [5]:
tokenizer

GPT2TokenizerFast(name_or_path='EleutherAI/gpt-neo-125m', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<|pad|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [6]:
torch.cuda.memory_allocated()

555601920

In [7]:
dataset = PersonalDataset(descriptions, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
Trainer(model=model, args=training_args, train_dataset=train_dataset,
       eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

[2023-06-15 00:48:23,478] [WARNING] [cpu_adam.py:84:__init__] FP16 params for CPUAdam may not work on AMD CPUs
 [WARNING]  cpu_adam cuda is missing or is incompatible with installed torch, only cpu ops can be compiled!


Using /home/butt_sahab/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/butt_sahab/.cache/torch_extensions/py38_cu117/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
Time to load cpu_adam op: 3.913334846496582 seconds


Loading extension module cpu_adam...
Using /home/butt_sahab/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/butt_sahab/.cache/torch_extensions/py38_cu117/utils/build.ninja...
Building extension module utils...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
Time to load utils op: 0.12505006790161133 seconds


Loading extension module utils...


Rank: 0 partition count [1] and sizes[(125200128, False)] 


Using /home/butt_sahab/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...


Time to load utils op: 0.0005056858062744141 seconds


Step,Training Loss


In [ ]:
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids.cuda()

In [ ]:
input_text = "<|startoftext|> Muqeeet, one of your closest freinds in the university. He had decided to help you set up a girl and went out with you and her to help smooth it along. But as it turns out he was sitting by her and talking/laughing while excluding you. You are sitting in KFC with muqeet and the girl nameed AX2. Both muqeet and AX2 are sitting together. You: I think we should switch seats right muqeet? Muqeet:"
encoded_input = tokenizer(input_text, return_tensors="pt").input_ids
sample_output=model.generate(encoded_input, 
                do_sample=True, 
                top_k=50,
                max_length=300, 
                top_p=0.5, 
                temperature=0.7,
                num_return_sequences=1)
output_text = tokenizer.decode(sample_output[0],skip_special_tokens=True)
output_text

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


/home/butt_sahab/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1070/2114095001.py:3 in <module>                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1070/2114095001.py'                         │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/utils/_contextlib.py:115 in decorate_context        │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /home/butt_sahab/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1485 in     │
│ generate                                                                                         │
│                                                                                                  │
│   1482 │   │   │   )                                                                             │
│   1483 │   │   │                                                                                 │
│   1484 │   │   │   # 13. run sample                                                              │
│ ❱ 1485 │   │   │   return self.sample(                                                           │
│   1486 │   │   │   │   input_ids,                                                                │
│   1487 │   │   │   │   logits_processor=logits_processor,                                        │
│   1488 │   │   │   │   logits_warper=logits_warper,                                              │
│                                                                                                  │
│ /home/butt_sahab/.local/lib/python3.8/site-packages/transformers/generation/utils.py:2524 in     │
│ sample                                                                                           │
│                                                                                                  │
│   2521 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2522 │   │   │                                                                                 │
│   2523 │   │   │   # forward pass to get next token                                              │
│ ❱ 2524 │   │   │   outputs = self(                                                               │
│   2525 │   │   │   │   **model_inputs,                                                           │
│   2526 │   │   │   │   return_dict=True,                                                         │
│   2527 │   │   │   │   output_attentions=output_attentions,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1501 in _call_impl             │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backwa

In [ ]:
model.save_pretrained("./Flaskapp/uo")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("./Flaskapp/uo")